# Khám phá mối quan hệ trong dữ liệu

## Import các thư viện cần thiết

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from wordcloud import WordCloud
import plotly.express as px


pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)

## Đọc dữ liệu từ file csv vào dataframe

In [2]:
data = pd.read_csv("../datasets/data_preprocess.csv", index_col = 0)
data.head()

,film_code,cinema_code,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,1492,304,3900000,26,0,4,4.26,150000.0,26,610.328638,2018-05-05,5,2,5
1,1492,352,3360000,42,0,5,8.08,80000.0,42,519.801980,2018-05-05,5,2,5
2,1492,489,2560000,32,0,4,20.00,80000.0,32,160.000000,2018-05-05,5,2,5
3,1492,429,1200000,12,0,1,11.01,100000.0,12,108.991826,2018-05-05,5,2,5
4,1492,524,1200000,15,0,3,16.67,80000.0,15,89.982004,2018-05-05,5,2,5


In [3]:
data['date'] = pd.to_datetime(data['date'])
data[['film_code', 'cinema_code']] = data[['film_code', 'cinema_code']].astype(str)

In [4]:
data

,film_code,cinema_code,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,1492,304,3900000,26,0,4,4.26,150000.0,26,610.328638,2018-05-05,5,2,5
1,1492,352,3360000,42,0,5,8.08,80000.0,42,519.801980,2018-05-05,5,2,5
2,1492,489,2560000,32,0,4,20.00,80000.0,32,160.000000,2018-05-05,5,2,5
3,1492,429,1200000,12,0,1,11.01,100000.0,12,108.991826,2018-05-05,5,2,5
4,1492,524,1200000,15,0,3,16.67,80000.0,15,89.982004,2018-05-05,5,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142519,1569,495,1320000,22,0,2,3.86,60000.0,22,569.948187,2018-11-04,11,4,4
142520,1569,474,1200000,15,0,1,65.22,80000.0,15,22.999080,2018-11-04,11,4,4
142521,1569,524,1060000,8,0,3,9.20,132500.0,8,86.956522,2018-11-04,11,4,4
142522,1569,529,600000,5,0,2,5.00,120000.0,5,100.000000,2018-11-04,11,4,4


### Số lượng các bộ phim được chiếu theo thời gian

In [74]:
n_film_by_date = data.groupby('date')['film_code'].nunique().reset_index()

In [81]:
fig = px.scatter(n_film_by_date, x='date', y='film_code', 
              title='Số lượng các bộ phim được chiếu trong ngày theo thời gian')

fig.update_layout(xaxis_title='thời gian',
                  yaxis_title='Số lượng')

fig.show()

In [77]:
n_film_by_month = data.groupby('month')['film_code'].nunique().reset_index()

In [83]:
fig = px.line(n_film_by_month, x='month', y='film_code', 
              title='Số lượng các bộ phim được chiếu trong tháng theo thời gian')

fig.update_layout(xaxis_title='tháng',
                  yaxis_title='Số lượng')

fig.show()

### Số suất chiếu phim theo thời gian của các rạp

In [84]:
n_show_time_by_date = data.groupby('date')['show_time'].sum().reset_index()

In [86]:
fig = px.scatter(n_show_time_by_date, x='date', y='show_time', 
              title='Số lượng suất chiếu phim trong ngày theo thời gian')

fig.update_layout(xaxis_title='thời gian',
                  yaxis_title='Số lượng')

fig.show()

In [92]:
n_show_time_by_week = data.groupby('date')['show_time'].sum().resample('W').sum().reset_index()

In [95]:
fig = px.line(n_show_time_by_week, x='date', y='show_time', 
              title='Số lượng suất chiếu phim trong ngày theo thời gian')

fig.update_layout(xaxis_title='thời gian',
                  yaxis_title='Số lượng')

fig.show()